Premier test pour le jeu Ghost in the cell
- strategie naive : 
à chaque tour on envoie une troupe (quantité de cyborg choisie par le joueur) dans l'usine la plus proche parmi toutes mes bases. 

In [ ]:
import sys
import math
from operator import itemgetter
from copy import deepcopy

factory_count = int(input())  # the number of factories
link_count = int(input())  # the number of links between factories

def get_near_link(base_id, mineList, excludeMe=0):
    print("gnl base_id", base_id, file=sys.stderr)
    available = []
    for link in link_list:
        link["case"] = 0
        if link["fact1"] == base_id:
            link["case"] = 1
        elif link["fact2"] == base_id:
            link["case"] = 2

        if link["case"] > 0:
            available.append(link)
    # On trie les liens de maniere croissante
    available.sort(key=itemgetter("distance"))

    i = 0 
    #si je compte le noeud courant je renovoi le premier lien 
    if excludeMe == 0:
        print("gnl return", available[i], file=sys.stderr)
        return (available[i])
    
    else:
     #exclue les bases m'appartenant des liens renvoyés
        tmp = i
        while tmp == i and i < len(available) - 1:
            #tant qu'il y a deux bases à moi et que ce n'est pas le dernier lien
            tmp += 1
            if link["case"] == 1:
                if mineList.count(available[i]["fact1"]) == 1:
                    i += 1
            else:
                if mineList.count(available[i]["fact2"]) == 1:
                    i += 1
            if i == len(available):
                break 
        print("gnl return", available[i], file=sys.stderr)
        return (available[i])

#initinialisation des differents links (usine1,usine2,ditance)
my_dict = {}
link_list = []
for i in range(link_count):
    my_dict = {}
    my_dict["fact1"], my_dict["fact2"], my_dict["distance"] = [int(j) for j in input().split()]
    link_list.append(my_dict)
    print("link:", my_dict, file=sys.stderr)

# game loop
while True:
    print("WAIT")
    my_bases = []
    entity_count = int(input())  
    print("entity_count", entity_count, file=sys.stderr)
    
    for i in range(entity_count):
        my_dict = {}
        entity_id, entity_type, arg_1, arg_2, arg_3, arg_4, arg_5 = input().split()
        my_dict["id"] = int(entity_id)
        my_dict["arg1"] = int(arg_1)
        my_dict["arg2"] = int(arg_2)
        my_dict["arg3"] = int(arg_3)
        my_dict["arg4"] = int(arg_4)
        my_dict["arg5"] = int(arg_5)
        if entity_type == "FACTORY":
            if my_dict["arg1"] == 1:
                my_bases.append(my_dict)

    print("MY_FACTORYES", file=sys.stderr)
    mineList = []
    for base in my_bases:
        mineList.append(base["id"])
    for base in my_bases:
        print(base , file=sys.stderr)
        near_link = get_near_link(base["id"], 1)
        print(near_link, file=sys.stderr)
        if near_link["fact1"] == base["id"]:
            print("MOVE", base["id"], near_link["fact2"], 10)
        else:
            print("MOVE", base["id"], near_link["fact1"], 10)

Modelisation de UCT/MonteCarlo

UCT en quatre étapes :

- la sélection de la branche : 
On part de la racine (l'état de la racine est notre état courant),on descend dans un noeud action selon la stratégie UCB (formule exploration/exploitation), puis on simule une transition du MDP (à partir de l'état courant et de l'action choisie) pour obtenir un nouvel état courant.

Si le noeud existe déjà parmi les fils du noeud action (et que l'état n'est pas un état terminal), alors on descend dans ce noeud et on recommence l'étape de sélection, sinon le noeud est à créer et on passe à l'étape suivante.

- la création du noeud : 
Si l'état courant est terminal, le noeud n'a pas besoin d'être développé, et on passe directement à l'étape suivante. 
Sinon, l'algorithme se trouve dans un noeud action qui ne possède pas l'état courant comme fils, on ajoute alors l'état courant comme nouveau fils du noeud action et on passe à l'étape suivante.
- la simulation de la politique par défaut : 
Dans cette étape, on cherche à estimer une valeur correspondant à la récompense que l'on peut espérer à partir de l'état du noeud que l'on vient de créer. Il faut disposer d'une politique par défaut, et simuler cette politique depuis l'état courant jusqu'à un état terminal et retenir les récompenses générées dans une
variable v.

- la mise à jour de l'arbre : 
    On remonte enfin les valeurs dans l'arbre de la façon suivante :
        quand on rencontre un noud action, on incrémente son compteur, on met à jour la variable v (c'est a dire on lui ajoute la récompense obtenue à ce noeud via la fonction de récompense) et on met à jour la valeur du noeud grâce à la variable v (la valeur du noeud correspond à la moyenne de toutes les valeurs des variables qui ont été remontées dans ce noeud).

Modelisation de UCT/MonteCarlo

Representation d'un etat : 
- liste des bases du joueurs 1 et 2 
- nombre de troupes dans chaque base
- distances pour chaque base
- nombre de cyborgs au total pour les deux joueurs

Actions : 
- MOVE usine1 usine2 quantité (envoyer une troupe d'une usine a une autre)

Recompense : 
- Conquete d'une base +
- Perte d'une base - 
- gagner la partie +


In [ ]:
class Node:
    def __init__(self, state, parent=None):
        self.state = state
        self.parent = parent
        self.children = []
        self.visits = 0
        self.valeur = 0


def simulate_tour(state,action)
    #simuler un tour a partir d'un etat a partir d'une action
    return new_state,recompense


def simulate_game(state,politique):
    #simuler toute une partie a partir d'un état et d'une politique, on va jusqu'à l'etat final
    return recompense_finale 


def get_all_actions(state):
    #retourne toutes les actions a partir d'un etat
    return liste_actions 


def ucb_score(parent_visits, node_wins, node_visits, coefficient_exploration=1.0):
    exploitation = node_wins / node_visits
    exploration = coefficient_exploration * math.sqrt(math.log(parent_visits) / node_visits)
    return exploitation + exploration

def select_best_child(node,coefficient_exploration):
    return max(node.children, key=lambda x: ucb_score(node.visits, x.value, x.visits, coefficient_exploration))

def backpropagate():
    #remonter les résultats de la simulation depuis le nœud sélectionné jusqu'à la racine de l'arbre.


def uct_monte_carlo(root_state, iterations=1000, coefficient_exploration=1.0):
    #algorithme de l'uct 
    return best_action